# Vergleich von Imputation Methoden

An dieser Stelle sollen verschiede Methoden zum interpolieren von fehlenden Werten betrachtet und verglichen werden.

### Vorbereitung

In [49]:
import pandas as pd
import numpy as np
import math

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error


In der Folge werden keine Regionen oder Gruppierungen von Ländern und nur die Jahre ab 1990 betrachtet. 
Von dem verbliebenen Datensatz werden nur jene Indikatoren behalten, die mindesten 20% gefüllt sind.

In [50]:
def reset_original():
    pd.set_option('display.float_format', lambda x: '%.4f' % x)

    base= pd.read_csv('../Data/WDIData.csv') #see downloads worldbank
    base = base.drop(['Country Code', 'Indicator Code', 'Unnamed: 66'], axis=1) #name of column 'Unnamed: 66' may differ

    countries = pd.read_csv('additional_data/countries.csv').drop('Unnamed: 0', axis=1)
    base = pd.merge(base, countries, how='left')
    base = base.loc[base['Type'] != 'Region'].drop('Type', axis=1)

    base = base.set_index(['Country Name', 'Indicator Name'])
    base = base.loc[:, '1990':'2020']

    idx = pd.IndexSlice
    keep = pd.DataFrame(pd.DataFrame(base.isna().groupby('Indicator Name').sum()).T.sum(), columns=['NaN'])
    keep = keep.loc[keep['NaN'] <len(base.index.get_level_values('Country Name').unique())*len(base.columns)*0.8] #kept if 80% of entries are not NaN
    base = base.loc[idx[:, keep.index], :]
    return base

In [51]:
def reset_base():
    base= pd.read_csv('additional_data/base.csv') 
    base.set_index(['Country Name', 'Indicator Name'], inplace=True)
    return base

In [52]:
base = reset_base()
base.isna().sum().sum()

170307

Um die Performanz unterschiedlicher Imutation Verfahren zu vergleichen werden weitere 1000 vorhandene (nicht NaN) Einträge entfernt. Diese 1000 Einträge werden später als Test-Daten verwendet, auf ihrer Grundlage lassen sich die Fehler der analysierten Verfahren errechnen. Um sicherzustellen, dass die Ergebnisse reproduzierbar sind wird ein Random State gesetzt. Dann werden zufällig Koordinaten zu Dateneinträgen gezogen. Da an dieser Stelle nur vorhandene Einträge relevant sind, werde zunächst zu viele Koordinaten gezogen, diese dann mit dem Datensatz abgeglichen und gelöscht, falls sie zu einem NaN zeigen und dann die ersten 1000 verbliebenen (und damit relevanten) Einträge ausgewählt. Diese werden in den Trainingsdaten entfernt. Auf diese Weise bleibt eine Reproduzierbarkeit erhalten.

In [53]:
base.head(3)

,,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Country Name,Indicator Name,,,,,,,,,,,,,,,,,,,,,
Afghanistan,Land area (sq. km),652860.0,652860.0,652860.0,652860.0,652860.0,652860.0,652860.0,652860.0,652860.0,652860.0,...,652860.0,652860.0,652860.0,652860.0,652860.0,652860.0,652860.0,652860.0,652860.0,652860.0
Albania,Land area (sq. km),27400.0,27400.0,27400.0,27400.0,27400.0,27400.0,27400.0,27400.0,27400.0,27400.0,...,27400.0,27400.0,27400.0,27400.0,27400.0,27400.0,27400.0,27400.0,27400.0,27400.0
Algeria,Land area (sq. km),2381740.0,2381740.0,2381740.0,2381740.0,2381740.0,2381740.0,2381740.0,2381740.0,2381740.0,2381740.0,...,2381740.0,2381740.0,2381740.0,2381740.0,2381741.0,2381741.0,2381741.0,2381741.0,2381741.0,2381741.0


### Simulation fehlender Werte und Evaluation

In [54]:
def get_cords(frac):
    n = int(base.isna().sum().sum()*frac)
    print(f'Testdaten mit {n}% fehlenden Werten (absolut: {n})')
    #random state to ensure reproducibility
    rnds = np.random.RandomState(n)

    #coordinates for data entries to be removed randomly
    #5000 entries are selected
    cords = pd.DataFrame([[rnds.randint(0, len(base), size=n*4)[i], 
                  rnds.randint(0, len(base.columns), size=n*4)[i]]
                  for i in range(n*4)])

    #all coordinates pointing to NaN entries are removed and
    #first 1000 remaining entries are selected
    cords['value'] = [base.iloc[cords[0][i], cords[1][i]] for i in cords.index]
    cords = cords.dropna()[:n].reset_index(drop=True)
    
    return cords

In [55]:
cords = get_cords(0.1)

Testdaten mit 17030% fehlenden Werten (absolut: 17030)


In [56]:
#getting train data by changing randomly chosen values to NaN
def reset_train():
    train = base.copy()
    for i in cords.index:
        train.iloc[cords[0][i], cords[1][i]] = None
    return train

In [57]:
def evaluate(df):
    #scaling original data and imputed data
    #necessary ?????????????????????????????????????
    res1 = (pd.DataFrame({'y_true': [base.iloc[cords[0][i], cords[1][i]] for i in cords.index],
                        'y_pred': [df.iloc[cords[0][i], cords[1][i]] for i in cords.index],
                        'indicator': [df.index.get_level_values('Indicator Name')[cords[0][i]] for i in cords.index],
                        'year': [df.columns[cords[1][i]] for i in cords.index]})
         )
    
    
    scaler = StandardScaler().fit(train) #fitting on train?
    norm_base = pd.DataFrame(scaler.transform(base))
    df = pd.DataFrame(scaler.transform(df))

    #getting imputed values for simulated NaNs and true value 
    res =pd.DataFrame({'y_true': [norm_base.iloc[cords[0][i], cords[1][i]] for i in cords.index],
                       'y_pred': [df.iloc[cords[0][i], cords[1][i]] for i in cords.index]
                      })
    res = res.dropna()

   
    #calculate evaluation metrics
    r2 = r2_score(res['y_true'], res['y_pred'])
    rmse = math.sqrt(mean_squared_error(res['y_true'], res['y_pred']))
    still_missing = df.isna().sum().sum()
    
    print(f'Mit dieser Methode bleiben {still_missing} NaNs bestehen.')
    print('')
    print(f'{len(res)} Werte wurden für die Metriken verwendet.')
    print(f'r2: {r2}, rmse: {rmse}')

    return still_missing, r2, rmse
    #return res1

###  Imputation Verfahren

Es werden diese Imputation Methoden verglichen:
- Backcasting
- Durchschnitt
- regionaler Durchschnitt

#### Backfill

In [58]:
def impute_backfill(df):
    df = df.fillna(method='bfill', limit=3)
    return df

In [59]:
base = reset_base()
train = reset_train()
df= impute_backfill(train) 

evaluate(df)

Mit dieser Methode bleiben 64593 NaNs bestehen.

16760 Werte wurden für die Metriken verwendet.
r2: -2.4092312597019343, rmse: 1.8190767598995974


(64593, -2.4092312597019343, 1.8190767598995974)

#### Durchschnitt des Indikators über alle Jahre hinweg

In [60]:
def impute_overall_means(df):
    #fill NaNs with overall mean of that indicator
    values = pd.DataFrame(df.stack()).groupby('Indicator Name')[0].mean()
    df = pd.DataFrame(df.stack(dropna=False))
    
    df[0] = df[0].fillna(df.groupby('Indicator Name')[0].transform('mean'))
    df = df.unstack()
    df.columns = df.columns.droplevel(0)
    df = df.sort_index(level='Indicator Name')
        
    return df

In [61]:
base = reset_base()
train = reset_train()
df = impute_overall_means(train)

evaluate(df)

Mit dieser Methode bleiben 0 NaNs bestehen.

17030 Werte wurden für die Metriken verwendet.
r2: -0.20295145686571625, rmse: 1.071967426854792


(0, -0.20295145686571625, 1.071967426854792)

#### Durchschnitt des Indikators für das jeweilige Jahr

In [62]:
def impute_yearly_means(df):
    #fill NaNs with overall mean of that indicator
    
    for i in df.columns:
        df[i] = df[i].fillna(df.groupby('Indicator Name')[i].transform('mean'))
            
    return df

In [63]:
base = reset_base()
train = reset_train()
df = impute_yearly_means(train)

evaluate(df)

Mit dieser Methode bleiben 52456 NaNs bestehen.

17029 Werte wurden für die Metriken verwendet.
r2: 0.0260213877002079, rmse: 1.0708498012053167


(52456, 0.0260213877002079, 1.0708498012053167)

#### Regionaler Durchschnitt des Indikators für das jeweilige Jahr

In [64]:
def impute_yearly_means_per_region(df):
    country_data = pd.read_csv('../Data/WDICountry.csv')
    country_data = country_data.loc[:,['Table Name', 'Region']]
    df = pd.merge(df.reset_index(), country_data, how='left', left_on='Country Name', right_on='Table Name').drop('Table Name', axis=1)
    df = df.set_index(['Country Name', 'Indicator Name', 'Region'])

    for i in df.columns:
        df[i] = df[i].fillna(df.groupby(['Indicator Name', 'Region'])[i].transform('mean'))

    df = df.reset_index().set_index(['Country Name', 'Indicator Name']).drop('Region', axis=1)
    return df

In [65]:
base = reset_base()
train = reset_train()
df = impute_yearly_means_per_region(train)
evaluate(df)

Mit dieser Methode bleiben 57345 NaNs bestehen.

16903 Werte wurden für die Metriken verwendet.
r2: 0.15676407843005447, rmse: 0.9008547931175253


(57345, 0.15676407843005447, 0.9008547931175253)

#### Interpolation

In [68]:
def interpolate3(df):
    df = df.interpolate(limit=3)
    return df

def interpolate5(df):
    df = df.interpolate(limit=5)
    return df

In [71]:
base = reset_base()
train = reset_train()
df = interpolate5(train)

evaluate(df)

Mit dieser Methode bleiben 58021 NaNs bestehen.

16970 Werte wurden für die Metriken verwendet.
r2: -1.125992036007828, rmse: 1.42759152496208


(58021, -1.125992036007828, 1.42759152496208)